In [4]:
import matplotlib.pyplot as plt
import pandas
import numpy as np
import random

# gerrychain imports
from gerrychain import (GeographicPartition, Partition, Graph, MarkovChain,
                        proposals, updaters, constraints, accept, Election)
from gerrychain.proposals import recom
from gerrychain.tree import recursive_tree_part
from gerrychain.accept import always_accept
from gerrychain.constraints import (Validator, single_flip_contiguous,
                                    within_percent_of_ideal_population, UpperBound)
from functools import partial

In [5]:
# State specific variables
state = "LA" # Louisiana
num_districts = 39 # senate districts
data_path = "./LA_data/LA_data.json"
pop_varname = "TOTPOP"
bvap_varname = "BVAP"
vap_varname = "VAP"
num_markov_steps = 20000 # in Markov chain
OPPORTUNITY_PERCENT = 0.4

In [6]:
# load the graph
graph = Graph.from_json(data_path)

In [7]:
#################################################################################
########################## Get ideal population  ################################
#################################################################################

def get_ideal_population(graph, pop_varname, num_districts):
    tot_pop = 0
    for i in range(len(graph.nodes)):
        tot_pop += graph.nodes[i][pop_varname]
        
    return int(tot_pop/num_districts)

ideal_population = get_ideal_population(graph, pop_varname, num_districts)

####################################################################################
############################# Define the updaters ##################################
####################################################################################

def count_opportunity_districts(partition):
    opp_percent = OPPORTUNITY_PERCENT
    
    num_opp_dists = 0
    for i in range(num_districts):
        num_bvap_percent = partition["bvap"][i] / partition["vap"][i]
        
        if num_bvap_percent >= opp_percent:
            num_opp_dists += 1

    return num_opp_dists

def compute_funky_score(partition):
    curr_bvap_list = []
    
    for i in range(num_districts):
        curr_bvap_percent = partition["bvap"][i] / partition["vap"][i]
        curr_bvap_list.append(curr_bvap_percent)
    
    curr_bvap_list = sorted(curr_bvap_list)
    
    curr_funky_score = 0
    for i in range(num_districts-1,-1,-1):
        if curr_bvap_list[i] >= OPPORTUNITY_PERCENT:
            curr_funky_score += 1
        else:
            curr_funky_score += curr_bvap_list[i]
            break
    
    return curr_funky_score

####################################################################################
###################### End of updater definitions ##################################
####################################################################################

updaters = {"population": updaters.Tally(pop_varname, alias="population"),
            "bvap":updaters.Tally(bvap_varname, alias='bvap'),
            "vap":updaters.Tally(vap_varname, alias='vap'),
            "opportunity_districts":count_opportunity_districts,
            "funky_score": compute_funky_score
           }

In [8]:
####################################################################################
################ Build Initial Partition and Markov Chain ##########################
####################################################################################

# assign each node to a district in a dict of the form {node: district}
seed_partition_dict = recursive_tree_part(graph, 
                                          range(num_districts), 
                                          pop_col=pop_varname,
                                          pop_target=ideal_population,
                                          epsilon=0.01, 
                                          node_repeats=1)

# use the partition dictionary to build a Partition object
seed_partition = GeographicPartition(graph, 
                                     assignment=seed_partition_dict, 
                                     updaters=updaters)

## Build the Markov chain
tree_proposal = partial(recom,
                        pop_col=pop_varname,
                        pop_target=ideal_population,
                        epsilon=0.05,
                        node_repeats=1)

# Constraints
popbound = within_percent_of_ideal_population(seed_partition, .1)

####################################################################################
###################### Define the acceptance functions #############################
####################################################################################

def increase_opportunity_districts(partition, bad_mobility_prob = 0.2):
    opp_percent = OPPORTUNITY_PERCENT
    if partition.parent is None:
        return True
    
    curr_opp_dists = 0
    prev_opp_dists = 0
    for i in range(num_districts):
        curr_bvap_percent = partition["bvap"][i] / partition["vap"][i]
        prev_bvap_percent = partition.parent["bvap"][i] / partition.parent["vap"][i]
        
        if curr_bvap_percent >= opp_percent:
            curr_opp_dists += 1
        if prev_bvap_percent >= opp_percent:
            prev_opp_dists += 1
        
    if curr_opp_dists >= prev_opp_dists: 
        return True
    else:
        return random.random() < bad_mobility_prob
            

# Funky Weighted Opportunity Score
def increase_funky_opportunity_score(partition, bad_mobility_prob = 0.2):
    
    curr_bvap_list = []
    prev_bvap_list = []
    
    for i in range(num_districts):
        curr_bvap_percent = partition["bvap"][i] / partition["vap"][i]
        prev_bvap_percent = partition.parent["bvap"][i] / partition.parent["vap"][i]
        
        curr_bvap_list.append(curr_bvap_percent)
        prev_bvap_list.append(prev_bvap_percent)
    
    curr_bvap_list = sorted(curr_bvap_list)
    prev_bvap_list = sorted(prev_bvap_list)
    
    curr_bvap_score = 0
    for i in range(num_districts-1,-1,-1):
        if curr_bvap_list[i] >= OPPORTUNITY_PERCENT:
            curr_bvap_score += 1
        else:
            curr_bvap_score += curr_bvap_list[i]
            break
    
    prev_bvap_score = 0 
    for i in range(num_districts-1,-1,-1):
        if prev_bvap_list[i] >= OPPORTUNITY_PERCENT:
            prev_bvap_score += 1
        else:
            prev_bvap_score += prev_bvap_list[i]
            break
            
    if curr_bvap_score >= prev_bvap_score:   
        return True
    else:
        return random.random() < bad_mobility_prob
       

In [9]:
####################################################################################
################################## Short Burst #####################################
####################################################################################    

def short_burst_run(initial_partition, num_bursts, num_steps, accept_function, score_function):
    max_partition = initial_partition
    max_score = 0 # bhushan come back here later...
    
    scores = []

    for i in range(num_bursts):
        chain = MarkovChain(tree_proposal, 
                          Validator([popbound]), 
                          accept=accept_function,
                          initial_state=max_partition, 
                          total_steps=num_steps)

        for partition in chain:
            current_score = score_function(partition)
            if current_score >= max_score: # > or >= has implications, not sure which is best...
                max_partition = partition
                max_score = current_score
            
            scores.append(current_score)
            # TODO: nasty code that needs to be rewritten
            seen_partition = initial_partition
            current_partition = partition
            seen_list = seen_partition.parts.values()
            seen_list = [sorted(list(unit)) for unit in seen_list]
            seen_list = sorted(seen_list)
        
            curr_list = current_partition.parts.values()
            curr_list = [sorted(list(unit)) for unit in curr_list]
            curr_list = sorted(curr_list)
        
            if seen_list == curr_list:
                print("same partition")
            
            
#         if i % 10 == 0:
#             print("Burst no. ", i)
                
    return scores


In [10]:
burst_lengths = [1]
num_bursts = [int(num_markov_steps / burst_length) for burst_length in burst_lengths]

for i in range(len(burst_lengths)):
    print("Burst Length: ", burst_lengths[i], ", Num bursts: ", num_bursts[i])
    scores = short_burst_run(seed_partition, 
                             num_bursts[i], 
                             burst_lengths[i],
                             always_accept, 
                             count_opportunity_districts)
    
    # save array 
    scores = np.array(scores)
    np.save("op_dist_counts_bl_" + str(burst_lengths[i]), scores)

Burst Length:  1 , Num bursts:  20000
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
sa

same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same parti

same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same parti

same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same parti

same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same parti

...

same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same parti

same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same parti

same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same parti

same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same parti

same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same partition
same parti

In [ ]:
# Opportunity district cell
recom_chain = MarkovChain(tree_proposal, 
                          Validator([popbound]), 
                          accept=increase_opportunity_districts,
                          initial_state=seed_partition, 
                          total_steps=num_markov_steps)

opportunity_districts_counts = []

for partition in recom_chain:
    opportunity_districts_counts.append(partition["opportunity_districts"])
    
plt.plot(opportunity_districts_counts, range(num_markov_steps))
plt.title("Opportunity districts increase / remain same")
plt.xlabel("# of opportunity districts")
plt.ylabel("# of steps in chain")

# save array 
opportunity_districts_counts = np.array(opportunity_districts_counts)
np.save("opportunity_district_counts", opportunity_districts_counts)

In [ ]:
# Funky score cell
recom_chain = MarkovChain(tree_proposal, 
                          Validator([popbound]), 
                          accept=increase_funky_opportunity_score,
                          initial_state=seed_partition, 
                          total_steps=num_markov_steps)

funky_scores = []

for partition in recom_chain:    
    funky_scores.append(partition["funky_score"])      

plt.plot(funky_scores, range(num_markov_steps))
plt.title("Funky Scores increase / remain same")
plt.xlabel("Funky Score")
plt.ylabel("# of steps in chain")

# save array 
funky_scores = np.array(funky_scores)
np.save("funky_scores", funky_scores)